In [ ]:
### LDA
import gensim
import gensim.corpora as corpora
import nltk
from nltk.corpus import stopwords
import pandas as pd 
# import modin.pandas as pd
# import ray
# ray.init(num_cpus=8)
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import re
import spacy
import string 
from gensim.corpora import Dictionary
from gensim.test.utils import get_tmpfile
import tqdm as tqdm
import glob as glob
import numpy as np
data_lemmatized=np.array(data_lemmatized)
np.save('data_lemmatized.npy',data_lemmatized) # Save as .npy format
a=np.load('data_lemmatized.npy',allow_pickle=True)
a=a.tolist()
data_lemmatized=a
def choose_topic(id2word,corpus,texts):#Choose the most suitable number of topics: lower perplexity and higher coherence is better
    '''
    @description: Generate models
    @param 
    @return: Generate LDA topic models with 1-15 topics and save them.
    '''
    id2word = Dictionary.load_from_text("id2word.txt")#Load generated dictionary
    corpus = corpora.MmCorpus('ths_corpuse.mm')  # Load
    texts = data_lemmatized
    coherence_values = []
    perplexity_values = []
    model_list = []
    for i in range(2,20,2):
        print('Current number of topics:{}'.format(i))
        print('Current data size:{}'.format(len(texts)))
        temp = 'lda_{}_{}'.format(i,len(texts))
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=id2word, passes=20)
        file_path = './{}.model'.format(temp)
        model.save(file_path)

        coherence_model_lda = CoherenceModel(model=model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)

        perplexity_values.append(model.log_perplexity(corpus))
        print('\nPerplexity: ', model.log_perplexity(corpus))

        coherence_values.append(coherence_model_lda.get_coherence())
        
        print('------------------------------')
    return model_list, coherence_values, perplexity_values
#### Based on perplexity and coherence score, choose 15 as the optimal number of topics and plot topic distribution for five drugs
data = pd.read_csv("/data1/data_8t/lwx/Drugtweets/药物分布/ivermectin.csv",low_memory=False, keep_default_na=False, lineterminator='\n')#Original file
data['topic'] = topic
data['p'] = p
data=data[['full_text','topic','p']]

for i in data[(data['topic']== 2) & (data['p']>0.7)]['full_text']: #View tweets corresponding to the topic
    print(i)
    print('--------------------------')


In [ ]:
##### Plotting
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # For displaying Chinese labels normally
plt.rcParams['axes.unicode_minus'] = False    # For displaying minus signs normally
# Number of coordinates on x-axis, also the number of elements in the list within data list. 4 can also be replaced with len(data[0])
X = np.arange(15)
fig = plt.figure(figsize=(15,28),dpi=300)
ax = fig.add_axes([0.1,0.1,0.8,0.8])
plt.xticks(fontsize=25)
plt.yticks(fontsize=30)
colors= ["#91D1C2","#F39B7F","#E64B35","#7E6148","#8491B4","#4DBBD5","#3C5488","#8491B4","#91D1C2","#DC0000","#7E6148"]
'''
F1, F2, F3 draw bar charts, three in total. The first parameter can be considered as labels, 
the second parameter is their respective y-coordinates. If we want to set x-axis scale values 
under a certain bar chart, we can use tick_label to set it under that bar chart.
'''
F5 = ax.barh(X + 0.00, ldaplt.iloc[:,5], color = colors[4], height = 0.18)
F4 = ax.barh(X + 0.18, ldaplt.iloc[:,4], color = colors[3], height = 0.18)
F3 = ax.barh(X + 0.36, ldaplt.iloc[:,3], color = colors[2], height = 0.18,tick_label=ldaplt['topics'])
F2 = ax.barh(X + 0.54, ldaplt.iloc[:,2], color = colors[1], height = 0.18)
F1 = ax.barh(X + 0.72, ldaplt.iloc[:,1], color = colors[0], height = 0.18)

ax.legend((F1,F2,F3,F4,F5),('Ivermectin','Hydroxychloroquine','Remdesivir','Zinc','Vitamin D'),fontsize=30)

plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel('Percentage of topics',fontsize=30)
plt.ylabel('Topics',fontsize=30)
fig.savefig('shu2.tif', bbox_inches = 'tight',transparent=True,dpi=300)
plt.show()